In [46]:
import os
import numpy as np
import torch
from torchvision.datasets import CIFAR10
from matplotlib import pyplot as plt
import cv2

In [47]:
download = not os.path.isdir('cifar-10-batches-py')
print(download)
d_train = CIFAR10(root='.', download=download, train=True)
d_test = CIFAR10(root='.', train=False)

x_train, y_train = d_train.data, d_train.targets
x_test, y_test = d_test.data, d_train.targets

False


In [48]:
num_train, num_test = 50000, 10000
height, width = 32, 32
print(x_train.shape)

(50000, 32, 32, 3)


In [61]:
def gray_images_YCrCb(dset, index):
    images = []
    
    image = np.zeros((height, width), dtype=np.uint8)
    for i in dset:
        
        red = i[:,:,0]
        green = i[:, :, 1]
        blue = i[:, :, 2]

        image = 0.3*red + 0.59*green + 0.11*blue
        
        images.append(image)
    
    return images

def gray_images_HSI(dset, index):
    images = []
    
    image = np.zeros((height, width), dtype=np.uint8)
    for i in dset:
        
        red = i[:,:,0]
        green = i[:, :, 1]
        blue = i[:, :, 2]

        image = (red + green + blue)/3
        
        images.append(image)
    
    return images

def gray_images_HSL(i, index):
    
    image = np.zeros((height, width), dtype=np.uint8)        
    for k in range(height):
        for j in range(width):
            red = i[k][j][0]
            green = i[k][j][1]
            blue = i[k][j][2]

            max_rgb = max(red, green, blue)
            min_rgb = min(red, green, blue)
            image[k][j] = (max_rgb+min_rgb)/2
    
    return image


def gray_images_HSV(i, index):
    
    image = np.zeros((height, width), dtype=np.uint8)
        
    for k in range(height):
        for j in range(width):
            H = 0.0
            S = 0.0
            V = 0.0
            red = i[k, j, 0]/100
            green = i[k, j, 1]/100
            blue = i[k, j, 2]/100


            min_rgb = min(red, green, blue)
            max_rgb = max(red, green, blue)

            V = max_rgb*100
            S = 0.0 if (max_rgb == 0) else (1 - (min_rgb/max_rgb))*100

            if(max_rgb == min_rgb): H = 0
            elif (max_rgb == red and green >= blue): H = 60*((green - blue)/(max_rgb - min_rgb))
            elif (max_rgb == red and green < blue): H = 60*((green - blue)/(max_rgb - min_rgb)) + 360
            elif (max_rgb == green): H = 60*((blue - red)/(max_rgb - min_rgb)) + 120
            elif (max_rgb == blue): H = 60*((red - green)/(max_rgb - min_rgb)) + 240

            image[k][j] = (H + S + V)/3
            
    return image

In [50]:
img_train_filter1 = gray_images_YCrCb(x_train, num_train)
img_test_filter1 = gray_images_YCrCb(x_test, num_test)
print("1")

1


In [51]:
img_train_filter2 = gray_images_HSI(x_train, num_train)
img_test_filter2 = gray_images_HSI(x_test, num_test)
print("2")

2


In [63]:
os.makedirs('dataset/train/filter1', exist_ok=True)
os.makedirs('dataset/train/filter2', exist_ok=True)
os.makedirs('dataset/train/filter3', exist_ok=True)
os.makedirs('dataset/train/filter4', exist_ok=True)
os.makedirs('dataset/test/filter1_test', exist_ok=True)
os.makedirs('dataset/test/filter2_test', exist_ok=True)
os.makedirs('dataset/test/filter3_test', exist_ok=True)
os.makedirs('dataset/test/filter4_test', exist_ok=True)
os.makedirs('dataset/train/y_train', exist_ok=True)
os.makedirs('dataset/test/y_test', exist_ok=True)

In [53]:
for i in range(len(img_train_filter1)):
    cv2.imwrite(f'dataset/train/filter1/filter1_{i}.jpg', img_train_filter1[i])

In [54]:
for i in range(len(img_train_filter2)):
    cv2.imwrite(f'dataset/train/filter2/filter2_{i}.jpg', img_train_filter2[i])

In [56]:
pred_img = np.zeros((height, width), dtype=np.uint8)
for i in range(len(x_train)):
    img = gray_images_HSL(x_train[i], num_train)
    if img.any() != pred_img.any():
        cv2.imwrite(f'dataset/train/filter3/filter3_{i}.jpg', img)
        pred_img = img
    else:
        cv2.imwrite(f'dataset/train/filter3/filter3_{i}.jpg', img_train_filter2[i])

C:\Users\schir\AppData\Local\Temp\ipykernel_15952\3192759219.py:44: RuntimeWarning: overflow encountered in ubyte_scalars
  image[k][j] = (max_rgb+min_rgb)/2


In [67]:
pred_img = np.zeros((height, width), dtype=np.uint8)
for i in range(len(x_train)):
    img = gray_images_HSV(x_train[i], num_train)
    if img.any() != pred_img.any():
        cv2.imwrite(f'dataset/train/filter4/filter4_{i}.jpg', img)
        pred_img = img
    else:
        cv2.imwrite(f'dataset/train/filter4/filter4_{i}.jpg', img_train_filter2[i])

In [68]:
for i in range(len(img_test_filter1)):
    cv2.imwrite(f'dataset/test/filter1_test/filter1_{i}.jpg', img_test_filter1[i])

In [69]:
for i in range(len(img_test_filter2)):
    cv2.imwrite(f'dataset/test/filter2_test/filter2_{i}.jpg', img_test_filter2[i])

In [66]:
pred_img = np.zeros((height, width), dtype=np.uint8)
for i in range(len(x_test)):
    img = gray_images_HSL(x_test[i], num_test)
    if img.any() != pred_img.any():
        cv2.imwrite(f'dataset/test/filter3_test/filter3_{i}.jpg', img)
        pred_img = img
    else:
        cv2.imwrite(f'dataset/test/filter3_test/filter3_{i}.jpg', img_test_filter2[i])

C:\Users\schir\AppData\Local\Temp\ipykernel_15952\301644046.py:44: RuntimeWarning: overflow encountered in ubyte_scalars
  image[k][j] = (max_rgb+min_rgb)/2


In [71]:
pred_img = np.zeros((height, width), dtype=np.uint8)
for i in range(len(x_test)):
    img = gray_images_HSV(x_test[i], num_test)
    if img.any() != pred_img.any():
        cv2.imwrite(f'dataset/test/filter4_test/filter4_{i}.jpg', img)
        pred_img = img
    else:
        cv2.imwrite(f'dataset/test/filter4_test/filter4_{i}.jpg', img_test_filter2[i])

In [64]:
for i in range(len(x_train)):
    cv2.imwrite(f'dataset/train/y_train/y_train{i}.jpg', x_train[i])
for i in range(len(x_test)):
    cv2.imwrite(f'dataset/test/y_test/y_test{i}.jpg', x_test[i])